In [4]:
from tsfresh import select_features, extract_features
from tsfresh.utilities.dataframe_functions import impute
from src.modules import conf, fit, raw_data, preprocess, df_columns
import pandas as pd
import numpy as np

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
data = fit.load_pcls(
        'zahradnik',
        'running',
        conf["Paths"]["pcl"],
    )
data = pd.concat(data)

100%|██████████| 361/361 [00:00<00:00, 3442.50it/s]


In [7]:
drop_columns = [x for x in data.columns if x not in df_columns]

In [23]:
data = data[['heart_rate','enhanced_speed']]

In [54]:
data = data[df_columns]

In [55]:
data.reset_index(inplace=True)

In [11]:
data

,timestamp,heart_rate,enhanced_speed,distance,enhanced_altitude,cadence,temp,wind_speed,wind_direct,rain,slope_steep,slope_ascent,slope_descent
0,2021-11-22 08:05:16,85.0,6.5160,2.00,249.4,56.0,5.3,8.7,320.0,0.5,0.000000,0.0,0.0
1,2021-11-22 08:05:22,87.0,11.1528,23.28,249.4,86.0,5.3,8.7,320.0,0.5,0.000000,0.0,0.0
2,2021-11-22 08:05:26,90.0,14.6448,39.91,249.4,86.0,5.3,8.7,320.0,0.5,0.000000,0.0,0.0
3,2021-11-22 08:05:29,93.0,14.1084,52.04,249.0,86.0,5.3,8.7,320.0,0.5,3.297609,0.0,0.4
4,2021-11-22 08:05:30,96.0,13.1328,56.29,249.2,85.0,5.3,8.7,320.0,0.5,4.705882,0.2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
725602,2022-11-01 07:29:50,161.0,14.5800,11184.18,245.2,90.0,12.6,4.8,307.0,1.4,4.950495,0.2,0.0
725603,2022-11-01 07:29:51,160.0,14.5440,11188.23,245.4,90.0,12.6,4.8,307.0,1.4,4.938272,0.2,0.0
725604,2022-11-01 07:29:52,161.0,14.6124,11192.28,245.4,91.0,12.6,4.8,307.0,1.4,0.000000,0.0,0.0
725605,2022-11-01 07:29:53,162.0,14.6448,11196.35,245.8,90.0,12.6,4.8,307.0,1.4,9.828010,0.4,0.0


In [56]:
print(data.isnull().values.any())
print(data.isin([np.inf, -np.inf]).values.any())
print(data.isna().sum().any())

False
False
False


In [26]:
data['heart_rate'] = [int(x) for x in data.heart_rate]
data['enhanced_speed'] = [int(x) for x in data.enhanced_speed]

In [63]:
data['timestamp'] = [x for x in range(len(data))]

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
data =data[0:100]

In [58]:
for x in data:
    data[x] = [int(y) for y in data[x]]

In [37]:
print(len(data))
data.dropna(inplace=True)
print(len(data))

100
100


In [60]:
data = data[['heart_rate','cadence','enhanced_speed']]

In [64]:
data

,heart_rate,cadence,enhanced_speed,timestamp
0,85,56,6,0
1,87,86,11,1
2,90,86,14,2
3,93,86,14,3
4,96,85,13,4
...,...,...,...,...
725602,161,90,14,725602
725603,160,90,14,725603
725604,161,91,14,725604
725605,162,90,14,725605


In [ ]:
from tsfresh.feature_extraction import extract_features, MinimalFCParameters
extracted_features = extract_features(data, column_id="enhanced_speed",column_sort="timestamp",n_jobs=0)

Feature Extraction:  17%|█▋        | 10/60 [04:29<22:40, 27.21s/it]

In [16]:
extracted_features

,heart_rate__sum_values,heart_rate__median,heart_rate__mean,heart_rate__length,heart_rate__standard_deviation,heart_rate__variance,heart_rate__root_mean_square,heart_rate__maximum,heart_rate__absolute_maximum,heart_rate__minimum,...,slope_descent__sum_values,slope_descent__median,slope_descent__mean,slope_descent__length,slope_descent__standard_deviation,slope_descent__variance,slope_descent__root_mean_square,slope_descent__maximum,slope_descent__absolute_maximum,slope_descent__minimum
3.0060,532.0,134.5,133.000000,4.0,2.915476,8.50000,133.031951,135.0,135.0,128.0,...,1.0,0.2,0.250000,4.0,0.217945,0.047500,0.331662,0.6,0.6,0.0
3.0132,111.0,111.0,111.000000,1.0,0.000000,0.00000,111.000000,111.0,111.0,111.0,...,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
3.0240,11918.0,121.0,125.452632,95.0,18.591321,345.63723,126.822711,179.0,179.0,98.0,...,10.4,0.0,0.109474,95.0,0.329297,0.108437,0.347017,2.0,2.0,0.0
3.0276,430.0,110.5,107.500000,4.0,6.103278,37.25000,107.673116,112.0,112.0,97.0,...,0.2,0.0,0.050000,4.0,0.086603,0.007500,0.100000,0.2,0.2,0.0
3.0312,95.0,95.0,95.000000,1.0,0.000000,0.00000,95.000000,95.0,95.0,95.0,...,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30.6000,143.0,143.0,143.000000,1.0,0.000000,0.00000,143.000000,143.0,143.0,143.0,...,0.2,0.2,0.200000,1.0,0.000000,0.000000,0.200000,0.2,0.2,0.2
30.8376,147.0,147.0,147.000000,1.0,0.000000,0.00000,147.000000,147.0,147.0,147.0,...,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
30.9024,142.0,142.0,142.000000,1.0,0.000000,0.00000,142.000000,142.0,142.0,142.0,...,0.0,0.0,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
33.5916,148.0,148.0,148.000000,1.0,0.000000,0.00000,148.000000,148.0,148.0,148.0,...,0.2,0.2,0.200000,1.0,0.000000,0.000000,0.200000,0.2,0.2,0.2
